# Business Requirements and Data
Before you start working on any AI project, you need to understand the problem that you want to solve and make a plan for how you’re going to solve it. This involves 
- clearly defining the problem
- gathering requirements
- understanding the data and technology available to you
- setting clear expectations with stakeholders.
## Understand the Problem and Requirements
The need: stakeholders would like a system that answers to ad-hoc questions about patients, visits, physicians, hospitals, and insurance payers without having to understand a query language like SQL, request a report from an analyst, or wait for someone to build a dashboard.

Lets think Agent-like: we will gather a couple of questions and think wich tool could answer them 

Typical questions:
- What is the current wait time at XYZ hospital?
- Which hospital currently has the shortest wait time?
- At which hospitals are patients complaining about billing and insurance issues?
- Have any patients complained about the hospital being unclean?
- What have patients said about how doctors and nurses communicate with them?
- What are patients saying about the nursing staff at XYZ hospital?
- What was the total billing amount charged to Cigna payers in 2023?
- How many patients has Dr. John Doe treated?
- How many visits are open and what is their average duration in days?
- Which physician has the lowest average visit duration in days?
- How much was billed for patient 789’s stay?
- Which hospital worked with the most Cigna patients in 2023?
- What’s the average billing amount for emergency visits by hospital?
- Which state had the largest percent increase inedicaid visits from 2022 to 2023?

Tools to answer them:
- for budget&billing queries an sql query should be dinamically generated by the chatbot and pointed to the data
- for user reviews queries the chatbot should be able to manage them with a vector database and a retriever, just like in lesson 1

The agent should be able to distinguish between this two kind of questions and call the appropriate tool

## Explore the available data
Before building your chatbot, you need a thorough understanding of the data it will use to respond to user queries. This will help you determine what’s feasible and how you want to structure the data so that your chatbot can easily access it. All of the data you’ll use in this article was synthetically generated, and much of it was derived from a popular (health care dataset)[https://www.kaggle.com/datasets/prasad22/healthcare-dataset] on Kaggle.

In practice, the following datasets would likely be stored as tables in a SQL database, but you’ll work with CSV files to keep the focus on building the chatbot. They are all on ./data folder

### hospital.csv
A dimension table.
- hospital_id: An integer that uniquely identifies a hospital.
- hospital_name: The hospital’s name.
- hospital_state: The state the hospital is located in.

In [1]:
import polars as pl

HOSPITAL_DATA_PATH = "data/hospitals.csv"
data_hospitals = pl.read_csv(HOSPITAL_DATA_PATH)

print(data_hospitals.shape)
data_hospitals.head()

(30, 3)


hospital_id,hospital_name,hospital_state
i64,str,str
0,"""Wallace-Hamilton""","""CO"""
1,"""Burke, Griffin and Cooper""","""NC"""
2,"""Walton LLC""","""FL"""
3,"""Garcia Ltd""","""NC"""
4,"""Jones, Brown and Murray""","""NC"""


### physicians.csv
A dimension table.
- physician_id: An integer that uniquely identifies each physician.
- physician_name: The physician’s name.
- physician_dob: The physician’s date of birth.
- physician_grad_year: The year the physician graduated medical school.
- medical_school: Where the physician attended medical school.
- salary: The physician’s salary.


In [2]:
PHYSICIAN_DATA_PATH = "data/physicians.csv"
data_physician = pl.read_csv(PHYSICIAN_DATA_PATH)
print(data_physician.shape)
data_physician.head()

(500, 6)


physician_name,physician_id,physician_dob,physician_grad_year,medical_school,salary
str,i64,str,str,str,f64
"""Joseph Johnson""",0,"""1970-02-22""","""2000-02-22""","""Johns Hopkins University Schoo…",309534.155076
"""Jason Williams""",1,"""1982-12-22""","""2012-12-22""","""Mayo Clinic Alix School of Med…",281114.503559
"""Jesse Gordon""",2,"""1959-06-03""","""1989-06-03""","""David Geffen School of Medicin…",305845.584636
"""Heather Smith""",3,"""1965-06-15""","""1995-06-15""","""NYU Grossman Medical School""",295239.766689
"""Kayla Hunter DDS""",4,"""1978-10-19""","""2008-10-19""","""David Geffen School of Medicin…",298751.355201


### payers.csv
dimension table about the insurance companies that the hospitals bill.
- payer_id: An integer that uniquely identifies each payer.
- payer_name: The payer’s company name.

In [5]:
DATA_PATH = "data/payers.csv"
data_payers = pl.read_csv(DATA_PATH)
data_payers

payer_name,payer_id
str,i64
"""Medicaid""",0
"""UnitedHealthcare""",1
"""Aetna""",2
"""Cigna""",3
"""Blue Cross""",4


### reviews.csv
We know this one from the previous lesson. Contains patient reviews about their experience at the hospital. It has these fields:
- review_id: An integer that uniquely identifies a review.
- visit_id: An integer that identifies the patient’s visit that the review was about.
- review: This is the free form text review left by the patient.
- physician_name: The name of the physician who treated the patient.
- hospital_name: The hospital where the patient stayed.
- patient_name: The patient’s name.

In [6]:
DATA_PATH = "data/reviews.csv"
data_reviews = pl.read_csv(DATA_PATH)
print(data_reviews.shape)
data_reviews.head()

(1005, 6)


review_id,visit_id,review,physician_name,hospital_name,patient_name
i64,i64,str,str,str,str
0,6997,"""The medical staff at the hospi…","""Laura Brown""","""Wallace-Hamilton""","""Christy Johnson"""
9,8138,"""The hospital's commitment to p…","""Steven Watson""","""Wallace-Hamilton""","""Anna Frazier"""
11,680,"""The hospital's commitment to p…","""Chase Mcpherson Jr.""","""Wallace-Hamilton""","""Abigail Mitchell"""
892,9846,"""I had a positive experience ov…","""Jason Martinez""","""Wallace-Hamilton""","""Kimberly Rivas"""
822,7397,"""The medical team at the hospit…","""Chelsey Davis""","""Wallace-Hamilton""","""Catherine Yang"""


### visits.csv
This is the fact table. it logs every visit to the hospital by its dimensions: patient, doctor, hospital and payer. For each one it logs the data about reason of admission, test results, etc.

Note that some rows lack some of the data (chief complaint, treatment, ...) : thats real world for you

Here are the fields, ids for the dimensions and normal data for the rest:

- visit_id: The unique identifier of a hospital visit.
- patient_id: The ID of the patient associated with the visit.
- date_of_admission: The date the patient was admitted to the hospital.
- room_number: The patient’s room number.
- admission_type: One of ‘Elective’, ‘Emergency’, or ‘Urgent’.
- chief_complaint: A string describing the patient’s primary reason for being at the hospital.
- primary_diagnosis: A string describing the primary diagnosis made by the physician.
- treatment_description: A text summary of the treatment given by the physician.
- test_results: One of ‘Inconclusive’, ‘Normal’, or ‘Abnormal’.
- discharge_date: The date the patient was discharged from the hospital
- physician_id: The ID of the physician that treated the patient.
- hospital_id: The ID of the hospital the patient stayed at.
- payer_id: The ID of the insurance payer used by the patient.
- billing_amount: The amount of money billed to the payer for the visit.
- visit_status: One of ‘OPEN’ or ‘DISCHARGED’.

In [7]:
DATA_PATH = "data/visits.csv"
data_visits = pl.read_csv(DATA_PATH)
print(data_visits.shape)
data_visits.head()

(9998, 15)


patient_id,date_of_admission,billing_amount,room_number,admission_type,discharge_date,test_results,visit_id,physician_id,payer_id,hospital_id,chief_complaint,treatment_description,primary_diagnosis,visit_status
i64,str,f64,i64,str,str,str,i64,i64,i64,i64,str,str,str,str
0,"""2022-11-17""",37490.983364,146,"""Elective""","""2022-12-01""","""Inconclusive""",0,102,1,0,null,null,null,"""DISCHARGED"""
1,"""2023-06-01""",47304.064845,404,"""Emergency""",null,"""Normal""",1,435,4,5,null,null,null,"""OPEN"""
2,"""2019-01-09""",36874.896997,292,"""Emergency""","""2019-02-08""","""Normal""",2,348,2,6,null,null,null,"""DISCHARGED"""
3,"""2020-05-02""",23303.322092,480,"""Urgent""","""2020-05-03""","""Abnormal""",3,270,4,15,null,null,null,"""DISCHARGED"""
4,"""2021-07-09""",18086.344184,477,"""Urgent""","""2021-08-02""","""Normal""",4,106,2,29,"""Persistent cough and shortness…","""Prescribed a combination of in…","""J45.909 - Unspecified asthma, …","""DISCHARGED"""


### Wait times
We will use a "fake" wait time API to answer questions about that.


## Design the Chatbot
Your chatbot will use multiple tools to answer diverse questions about your hospital system. Here’s a flowchart illustrating how you’ll accomplish this:
![Architecture and data flow for the hospital system chatbot](architecture_chatbot.png)

This flowchart illustrates how data moves through your chatbot, starting from the user’s input query all the way to the final response. Here’s a summary of each component:

- LangChain Agent: The LangChain agent is the brain of your chatbot. Given a user query, the agent decides which tool to call and what to give the tool as input. The agent then observes the tool’s output and decides what to return to the user—this is the agent’s response.
- Neo4j AuraDB: You’ll store both structured hospital system data and patient reviews in a Neo4j AuraDB graph database. You’ll learn all about this in the next section.
- LangChain Neo4j Cypher Chain: This will be the data mining chain, fit for queries about billing ammounts, gains per doctor and so on so forth. This chain willñ try to convert the user query into Cypher, Neo4j’s query language, and execute the Cypher query in Neo4j. The chain then answers the user query using the Cypher query results. The chain’s response is fed back to the LangChain agent and sent to the user.
- LangChain Neo4j Reviews Vector Chain: This is very similar to the chain you built in Step 1, except now patient review embeddings are stored in Neo4j. The chain searches for relevant reviews based on those semantically similar to the user query, and the reviews are used to answer the user query.
- Wait Times Function: this will be the review mining chain. Similar to the logic in Step 1, the LangChain agent tries to extract a hospital name from the user query. The hospital name is passed as input to a Python function that gets wait times, and the wait time is returned to the agent.

To walk through an example, suppose a user asks *How many emergency visits were there in 2023?* The LangChain agent will receive this question and decide which tool (hospital waiting, data mining or review mining), if any, to pass the question to. In this case, the agent should pass the question to the *LangChain Neo4j Cypher Chain*. The chain will try to convert the question to a Cypher query, run the Cypher query in Neo4j, and use the query results to answer the question.

Once the LangChain Neo4j Cypher Chain answers the question, it will return the answer to the agent, and the agent will relay the answer to the user.

Queries about reviews or waiting time will be routed similary but by their own chain. 
